![Trolo](https://private-user-images.githubusercontent.com/15766192/391220869-08091cad-703d-4d7d-84f6-5e6c310bf6c2.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3MzMzMTMzOTksIm5iZiI6MTczMzMxMzA5OSwicGF0aCI6Ii8xNTc2NjE5Mi8zOTEyMjA4NjktMDgwOTFjYWQtNzAzZC00ZDdkLTg0ZjYtNWU2YzMxMGJmNmMyLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNDEyMDQlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjQxMjA0VDExNTEzOVomWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTQ4NDAwOTc2ZGNlMDQ1YmMwMWExMzBjYmE5NjJmMGYxMTE1NzY1ZDI0NGZkMTI5N2Q0YzRlMGJiMDc5ZDI2ZjgmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0In0.H8dvVZt5CImX2vEg2WP69FX3e3uAXIsuvo4TRjyQgE4)

# Object Tracking
---
[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ayushexel/trolo/blob/master/recipes/object-tracking.ipynb)

# Installation
---
#### Installing the Trolo package directly from the GitHub repo


In [ ]:
!pip install git+https://github.com/AyushExel/trolo.git

# Imports
---

In [ ]:
import numpy as np

from tqdm import tqdm
from PIL import Image

import supervision as sv
from supervision.assets import download_assets, VideoAssets

# Download Video
---
#### As an example, we will use a video from the `supervision.assets` library.

In [ ]:
import os
HOME = os.getcwd()

path_to_video = download_assets(VideoAssets.PEOPLE_WALKING)

In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/people-walking.mp4"

In [ ]:
video_info=sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# Initializing Objects
---
#### Initializing Predictor and Annotator objects

In [ ]:
from trolo.inference import DetectionPredictor

predictor = DetectionPredictor(model="dfine-s")
predictions = predictor.predict(SOURCE_VIDEO_PATH, return_inputs=True)

In [ ]:
bounding_box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)
byte_tracker = sv.ByteTrack()

# Running Tracking
---
#### Using `sv.VideoSink` to annotate detections and tracking onto the video frame by frame.

In [ ]:
with sv.VideoSink(target_path=f"/{HOME}/output.mp4", video_info=video_info) as sink:

    for prediction, frame in tqdm(predictions, total=video_info.total_frames):

        detections = sv.Detections(
            xyxy=prediction[0]['boxes'].numpy(),
            confidence=prediction[0]['scores'].numpy(),
            class_id=prediction[0]['labels'].numpy()
        )
        detections = byte_tracker.update_with_detections(detections)

        labels = [ f"{tracker_id}" for tracker_id in detections.tracker_id ]
        annotated_frame = label_annotator.annotate(scene=np.array(frame[0]), detections=detections, labels=labels)

        sink.write_frame(frame=annotated_frame)